In [6]:
# Ministry of Mines Auction Database URL
# url = 'https://mines.gov.in/webportal/content/archived/successful-auction-since-2015-6194' -- Depreciated
url = 'https://mines.gov.in/webportal/content/archived/sucessfully-auction-since-2015-6715'

In [7]:
# importing Library 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd
from datetime import datetime
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os
from dotenv import load_dotenv
import math

In [8]:
# triggering the chrome testing browser
chrome_options = Options()
chrome_options.use_chromium = True

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get(url)

In [9]:
headers = ['state_name','name_of_the_block','mineral','auction_date', 'ml_or_cl', 'area_in_hectare',
          'final_bid_in_per_cent','preferred_bidder']

In [10]:
def convert_date(date_str):
    try:
        date_obj = datetime.strptime(date_str, '%d.%m.%Y')
        return date_obj.strftime('%Y-%m-%d')
    except ValueError:
        try:
            date_obj = datetime.strptime(date_str, '%d-%m-%Y')
            return date_obj.strftime('%Y-%m-%d')
        except ValueError:
            date_obj = datetime.strptime(date_str, '%d%m.%Y.')
            return date_obj.strftime('%Y-%m-%d')

In [13]:
table_rows = driver.find_element(By.TAG_NAME,'tbody').find_elements(By.TAG_NAME, 'tr')
auction_data = []
for row in table_rows[3:]:
    cells = []
    data = row.find_elements(By.TAG_NAME,'td')[1:]
    
    if len(data) == 9:
        
        cells.append(data[0].text)
        cells.append(data[1].text)
        cells.append(data[2].text)
        
        # converting auction date from DD.MM.YYYY format to YYYY-MM-DD format
        x = convert_date(data[3].text)
        cells.append(x)
        cells.append(data[4].text)
        try:
            # Try to convert the text to a float
            value = float(data[5].text)
            cells.append(value)
        except ValueError:
            # If conversion fails, append NaN
            cells.append(np.nan)
        
        # try:
        #     # Try to convert the text to a float
        #     value = float(data[6].text)
        #     cells.append(value)
        # except ValueError:
        #     # If conversion fails, append NaN
        #     cells.append(np.nan)
        cells.append(float(data[7].text))
        cells.append(data[8].text)
        
        # cells = [data.text for data in row.find_elements(By.TAG_NAME,'td')[1:]]
        auction_data.append(cells)

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=131.0.6778.205)
Stacktrace:
0   chromedriver                        0x0000000104d4f184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000104d479d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x00000001047b4968 cxxbridge1$string$len + 89228
3   chromedriver                        0x000000010479e8d4 core::str::slice_error_fail::ha0e52dbcb60e6bae + 63828
4   chromedriver                        0x000000010479e804 core::str::slice_error_fail::ha0e52dbcb60e6bae + 63620
5   chromedriver                        0x0000000104831c40 cxxbridge1$string$len + 601956
6   chromedriver                        0x00000001047ed568 cxxbridge1$string$len + 321676
7   chromedriver                        0x00000001047ee1b8 cxxbridge1$string$len + 324828
8   chromedriver                  

In [ ]:
row.text

In [ ]:
df = pd.DataFrame(auction_data, columns = headers)

In [ ]:
df.head(5)

In [ ]:
# Save it in postgres database

from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os
from dotenv import load_dotenv
import math

# Load environment variables
load_dotenv(override=True)

# Define the PostgreSQL connection string
connection_string = os.getenv('POSTGRES_DB_URL')

# Create the SQLAlchemy engine for PostgreSQL
engine = create_engine(connection_string)

# Define the base for the declarative mapping
Base = declarative_base()

# Define the table schema for PostgreSQL
class AuctionData(Base):
    __tablename__ = 'mom_auction'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Auto-incrementing ID
    state_name = Column(String(255))
    name_of_the_block = Column(String(255))
    mineral = Column(String(255))
    auction_date = Column(Date)
    ml_or_cl = Column(String(10))
    area_in_hectare = Column(Float)
    final_bid_in_per_cent = Column(Float)
    preferred_bidder = Column(String(255))

# Create the table in the PostgreSQL database
Base.metadata.create_all(engine)

# Prepare your DataFrame (scraped data already processed in df)

# Define a session to interact with the PostgreSQL database
Session = sessionmaker(bind=engine)
session = Session()

# Convert the DataFrame to a list of dictionaries, compatible with SQLAlchemy ORM
auction_data_dicts = df.to_dict(orient='records')

# Insert data into the auction_data table
for record in auction_data_dicts:
    # Replace nan with None for PostgreSQL compatibility
    record = {key: (None if (isinstance(value, float) and math.isnan(value)) else value) 
              for key, value in record.items()}
    
    auction_entry = AuctionData(**record)  # Unpack the dictionary into the ORM object
    session.add(auction_entry)

# Commit the transaction
session.commit()

print("Data inserted successfully into PostgreSQL!")

# Close the session
session.close()